In [3]:
import numpy as np
import random

In [8]:
def _sample_pids(collection, typical_doclen=120):
    num_passages = len(collection)
    sampled_pids = 16 * np.sqrt(typical_doclen * num_passages)
    sampled_pids = min(1 + int(sampled_pids), num_passages)
    sampled_pids = random.sample(range(num_passages), sampled_pids)
    print(f"# of sampled PIDs = {len(sampled_pids)} \t sampled_pids[:3] = {sampled_pids[:3]}")
    return set(sampled_pids)

In [17]:
import torch
import os
def _sample_embeddings(sampled_pids, collection, index_path):
        local_sample = [passage for pid, passage in collection if pid in sampled_pids]
        local_sample_embs, doclens = encode_passages(local_sample)

        if torch.cuda.is_available():
            if torch.distributed.is_available() and torch.distributed.is_initialized():
                num_sample_embs = torch.tensor([local_sample_embs.size(0)]).cuda()
                torch.distributed.all_reduce(num_sample_embs)

                avg_doclen_est = sum(doclens) / len(doclens) if doclens else 0
                avg_doclen_est = torch.tensor([avg_doclen_est]).cuda()
                torch.distributed.all_reduce(avg_doclen_est)

            else:
                num_sample_embs = torch.tensor([local_sample_embs.size(0)]).cuda()
                avg_doclen_est = sum(doclens) / len(doclens) if doclens else 0
                avg_doclen_est = torch.tensor([avg_doclen_est]).cuda()

        else:
            if torch.distributed.is_available() and torch.distributed.is_initialized():
                num_sample_embs = torch.tensor([local_sample_embs.size(0)]).cpu()
                torch.distributed.all_reduce(num_sample_embs)

                avg_doclen_est = sum(doclens) / len(doclens) if doclens else 0
                avg_doclen_est = torch.tensor([avg_doclen_est]).cpu()
                torch.distributed.all_reduce(avg_doclen_est)

            else:
                num_sample_embs = torch.tensor([local_sample_embs.size(0)]).cpu()

                avg_doclen_est = sum(doclens) / len(doclens) if doclens else 0
                avg_doclen_est = torch.tensor([avg_doclen_est]).cpu()

        avg_doclen_est = avg_doclen_est.item()

        print(f'avg_doclen_est = {avg_doclen_est} \t len(local_sample) = {len(local_sample):,}')

        torch.save(local_sample_embs.half(), os.path.join(index_path, f'sample_embeddings.pt'))

        return avg_doclen_est

In [ ]:
# Saves sampled passages and embeddings for training k-means centroids later
sampled_pids = _sample_pids(collection, typical_doclen)
avg_doclen_est = _sample_embeddings(sampled_pids, collection, index_path)

# Select the number of partitions
num_passages = len(collection)
num_embeddings_est = num_passages * avg_doclen_est
num_partitions = int(2 ** np.floor(np.log2(16 * np.sqrt(num_embeddings_est))))

print(f'Creating {num_partitions:,} partitions.')
print(f'*Estimated* {int(num_embeddings_est):,} embeddings.')

In [2]:
!pip install faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 18.9 MB/s eta 0:00:00


In [3]:
import faiss

In [4]:
import torch
import numpy as np

# Define the dimensions
num_embeddings = 10000000
embedding_dim = 400
num_clusters = 800

# Generate cluster centers
cluster_centers = torch.randn(num_clusters, embedding_dim) * 10

# Generate random embeddings around these cluster centers
embeddings = []
for i in range(num_clusters):
    cluster_embeddings = cluster_centers[i] + torch.randn(num_embeddings // num_clusters, embedding_dim)
    embeddings.append(cluster_embeddings)
    print(i)

# Concatenate all embeddings
embeddings = torch.vstack(embeddings)

# If num_embeddings is not exactly divisible by num_clusters, add some extra embeddings
if len(embeddings) < num_embeddings:
    extra_embeddings = cluster_centers[-1] + torch.randn(num_embeddings - len(embeddings), embedding_dim)
    embeddings = torch.vstack([embeddings, extra_embeddings])

# Shuffle the embeddings to mix different clusters
embeddings = embeddings[torch.randperm(embeddings.size(0))]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [5]:
def compute_faiss_kmeans(dim, num_partitions, kmeans_niters, sample_embeddings):
    use_gpu = torch.cuda.is_available()
    kmeans = faiss.Kmeans(dim, num_partitions, niter=kmeans_niters, gpu=use_gpu, verbose=True, seed=123)
    sample = sample_embeddings.float().numpy()
    kmeans.train(sample)
    centroids = torch.from_numpy(kmeans.centroids)
    centroids = torch.nn.functional.normalize(centroids, dim=-1)
    if use_gpu:
      centroids = centroids.half()
    else:
      centroids = centroids.float()
    return centroids

In [7]:
compute_faiss_kmeans(400, 800, 20, embeddings)

tensor([[-0.0020, -0.0499,  0.0387,  ...,  0.0533, -0.0381, -0.0222],
        [ 0.0714, -0.0025, -0.0782,  ..., -0.0648, -0.0092, -0.0052],
        [-0.0975,  0.0592,  0.1016,  ..., -0.0480,  0.0019, -0.0415],
        ...,
        [ 0.0045, -0.0234, -0.0796,  ..., -0.0337,  0.0211, -0.0362],
        [ 0.0210,  0.0143, -0.0137,  ...,  0.0258, -0.0117,  0.0585],
        [ 0.0354,  0.0561,  0.0238,  ..., -0.0521, -0.0307,  0.0880]],
       dtype=torch.float16)